In [1]:
import findspark 
findspark.init()


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master("local[*]").getOrCreate()


2023-11-09 17:29:12,612 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#define the paths to the csv's
orders_df_path="file:/home/labuser/Downloads/orders.csv"
restaurants_df_path="file:/home/labuser/Downloads/restaurants.csv"

#read the dataframes
orders_df=spark.read.format("csv").option("header","true").load(orders_df_path)
restaurants_df=spark.read.format("csv").option("header","true").load(restaurants_df_path)

#add the year and month and join the dataframes and select only the desired columns
orders_df=orders_df.withColumn("year",year(col("order_date").cast("Date")))\
                   .withColumn("month",month(col("order_date").cast("Date")))
report=orders_df.join(restaurants_df,restaurants_df["restaurant_id"]==orders_df["restaurant_ids_orders"])\
         .select("year","month","cuisine","order_id")

#show the report
report=report.groupBy("year","month","cuisine").agg(count("order_id").alias("#orders"))\
             .orderBy(col("year"),col("month").desc())




#write to the disk the report with partitioning by year,month
report.write.option("header",True) \
        .partitionBy("year","month") \
        .mode("overwrite") \
        .csv("file:/home/labuser/Downloads/new_data")

#read the stored file
csv_data = spark.read.csv("file:/home/labuser/Downloads/new_data", header=True, inferSchema=True)


#repartition and save the csv 
csv_data.repartition(5).write.option("header",True).partitionBy("year","month").mode("overwrite").csv("file:/home/labuser/Downloads/new_data1")


In [4]:
#read the stored file
csv_data = spark.read.csv("file:/home/labuser/Downloads/new_data1", header=True, inferSchema=True)


#repartition and save the csv 
csv_data.coalesce(3).write.option("header",True).partitionBy("year","month").mode("overwrite").csv("file:/home/labuser/Downloads/new_data2")


In [5]:
orders_details_path_local="file:/home/labuser/Downloads/orders_details.csv"
food_path_local="file:/home/labuser/Downloads/food.csv"

In [6]:
# Q. Total Revenue of Veg Food Items

orders_details_df_local = spark.read.format("csv").option("header","true").option("inferSchema","true").load(orders_details_path_local)

food_df_local= spark.read.format("csv").option("header","true").option("inferSchema","true").load(food_path_local)

# Filter for vegetarian food items
vegetarian_food = food_df_local.filter(col("food_category") == "Veg")

# Join the data frames to get the relevant information
order_details_food = orders_details_df_local.join(vegetarian_food, orders_details_df_local.food_ids_detail == vegetarian_food.food_id)

# Calculate the revenue generated by vegetarian food items
revenue_df = order_details_food.select(sum(col("food_price") * col("food_qty")).alias("total_revenue"))

In [35]:
#define the path to the file
orders_details_path_local="file:/home/labuser/Downloads/orders_details.csv"

#read the data in to Dataframe
orders_details_df_local = spark.read.format("csv").option("header","true").option("inferSchema","true").load(orders_details_path_local)


#print the num of partitions after reading
num_partitions = orders_details_df_local.rdd.getNumPartitions()
print(num_partitions)

#repartiotion the dataframe 
orders_details_df_local=orders_details_df_local.coalesce(19)

#print the updated partitions count
print(orders_details_df_local.rdd.getNumPartitions())

8
8


In [33]:

#repartiotion the dataframe 
orders_details_df_local=orders_details_df_local.coalesce(10)

#print the updated partitions count
print(orders_details_df_local.getNumPartitions())

10


In [56]:
food_df_local.select("food_menu").distinct().show()

+--------------------+
|           food_menu|
+--------------------+
|   Chiness Appetizer|
|  Maggie & Pav Bhaji|
|         Cold Coffee|
|           Breakfast|
| Special Sweet Gifts|
|              Snacks|
|      Shakes Crunchy|
|         Quick Bites|
|   Veg Supreme Pizza|
|   Veg Delight Pizza|
|Single Topping Pizza|
|Non Veg Supreme P...|
|Non Veg Delight P...|
| Non Veg Super Pizza|
|Swiggy Special Co...|
|          Gravy Item|
|               Momos|
|Non-Veg Grilled S...|
|      Pasta-La-Vista|
|              Extras|
+--------------------+
only showing top 20 rows



In [50]:
orders_df.write.option("header",True) \
        .partitionBy("year","month") \
        .mode("overwrite") \
        .csv("file:/home/labuser/Downloads/new_data")